In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import pandas as pd
import re
import json
import h2o

from datetime import datetime

from utils import get_data, Config, clean_url, make_clickable, add_columns

from tree import make_tree

In [27]:
# Load all data (except unknown/manual testing)
# Original Desktop Runs+Re-Runs were performed between 2024-02-05 and 2024-02-12
# Additional browsers + more than one response_id per parsing URL was run between 2024-02-16 and TBD
initial_data = """
SELECT "Result".*, 
"Response".raw_header, "Response".status_code, "Response".label, "Response".resp_type,
"Browser".name, "Browser".version, "Browser".headless_mode, "Browser".os, "Browser".automation_mode, "Browser".add_info
FROM "Result"
JOIN "Response" ON "Result".response_id = "Response".id JOIN "Browser" ON "Result".browser_id = "Browser".id
WHERE "Browser".name != 'Unknown' AND "Result".created_at < '2024-02-17';
"""
df = get_data(Config(), initial_data)
df = add_columns(df)

Connecting to the PostgreSQL database...
Connection successful


In [28]:
df["raw_header"] = df["raw_header"].apply(bytes).apply(lambda x: json.loads(x.decode("utf-8"))).astype(str)

In [29]:
responses = """
SELECT * from "Response";
"""
responses = get_data(Config(), responses)

Connecting to the PostgreSQL database...
Connection successful


# Overview

In [30]:
# In total 2 runs + couple of "repeat"-mode runs such that all browsers have results for each test
# Safari has more results as we had to abort and restart the main run a couple of times
# Both basic + parsing mode
df.groupby(["automation_mode", "browser_id", "name", "version", "os", "headless_mode"])["id"].count().sort_values(ascending=False).to_frame().reset_index()

,automation_mode,browser_id,name,version,os,headless_mode,id
0,selenium,42,chrome,121,Ubuntu 22.04,headless-new,599532
1,selenium,43,firefox,122,Ubuntu 22.04,headless,527250
2,selenium,45,brave,v1.62.156 (121.0.6167.139),Ubuntu 22.04,headless-new,527107
3,selenium,44,edge,121,Ubuntu 22.04,headless-new,522142
4,selenium,46,safari,17.3,macOS 14.3,real,396825
5,selenium,47,chrome,122,Ubuntu 22.04,headless-new,167364
6,selenium,48,chrome,120,Ubuntu 22.04,headless-new,163736
7,selenium,49,firefox,121,Ubuntu 22.04,headless,145630


# Error + Timeout Analysis
- Only timeout exist, no other systematic failures
- Systematic timeout: TAO in Safari -> fixed
- Couple of random timeouts in other tests

In [31]:
# Timeout (2), Fail (1), Not-run (3)
filtered_df = df.groupby(["browser"])["test_status"].filter(lambda x: x.nunique() >= 2)
display(df[["browser", "test_status"]].loc[df.index.isin(filtered_df.index)].groupby(["browser"])["test_status"].value_counts().to_frame())

count
browser                                            test_status        
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... 0            527058
                                                   2                49
chrome Ubuntu 22.04 120 selenium headless-new      0            163731
                                                   2                 5
chrome Ubuntu 22.04 121 selenium headless-new      0            599503
                                                   2                29
edge Ubuntu 22.04 121 selenium headless-new        0            522065
                                                   2                77
firefox Ubuntu 22.04 121 selenium headless         0            145606
                                                   2                24
firefox Ubuntu 22.04 122 selenium headless         0            527223
                                                   2                27
safari macOS 14.3 17.3 selenium real               0            396664
                                                   2               161

In [32]:
# Teststatus == 2 (timeout)

# Rare timeouts in all browsers; Safari some more than others
display(df[df["test_status"] == 2]["browser"].value_counts().to_frame())

# Mostly 302 redirects for basic (oac_iframe, oac_window.open), referrer_iframe 
# perfAPI_img (only Safari, fixed!)
# Others quite rare
display(df.loc[(df["test_status"] == 2)].groupby(["resp_type", "status_code", "test_name"])["browser"].value_counts().to_frame().sort_values(by="count", ascending=False))

,count
browser,
safari macOS 14.3 17.3 selenium real,161
edge Ubuntu 22.04 121 selenium headless-new,77
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,49
chrome Ubuntu 22.04 121 selenium headless-new,29
firefox Ubuntu 22.04 122 selenium headless,27
firefox Ubuntu 22.04 121 selenium headless,24
chrome Ubuntu 22.04 120 selenium headless-new,5


count
resp_type status_code test_name             browser                                                  
basic     302         oac_iframe            safari macOS 14.3 17.3 selenium real                   40
          200         referrer_iframe       safari macOS 14.3 17.3 selenium real                   38
parsing   200         perfAPI_img           safari macOS 14.3 17.3 selenium real                   32
basic     302         referrer_iframe       safari macOS 14.3 17.3 selenium real                   26
                      oac_window.open       safari macOS 14.3 17.3 selenium real                   24
                      oac_iframe            brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...     20
parsing   200         framing_iframe        firefox Ubuntu 22.04 121 selenium headless             18
basic     302         oac_window.open       edge Ubuntu 22.04 121 selenium headless-new            16
parsing   200         framing_iframe        edge Ubuntu 22.04 121 selenium headless-new            14
basic     302         oac_window.open       brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...     13
parsing   200         fullscreen_iframe     edge Ubuntu 22.04 121 selenium headless-new            12
                      framing_iframe        firefox Ubuntu 22.04 122 selenium headless             12
basic     302         oac_iframe            edge Ubuntu 22.04 121 selenium headless-new            12
                      referrer_iframe       brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      9
          200         referrer_iframe       firefox Ubuntu 22.04 122 selenium headless              9
                      framing_embed         chrome Ubuntu 22.04 121 selenium headless-new           8
                      framing_object        chrome Ubuntu 22.04 121 selenium headless-new           8
                      oac_iframe            edge Ubuntu 22.04 121 selenium headless-new             8
                      framing_iframe        chrome Ubuntu 22.04 121 selenium headless-new           7
          302         referrer_iframe       edge Ubuntu 22.04 121 selenium headless-new             7
          200         referrer_iframe       edge Ubuntu 22.04 121 selenium headless-new             6
parsing   200         framing_iframe        brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      5
                      fullscreen_iframe     firefox Ubuntu 22.04 122 selenium headless              5
                                            chrome Ubuntu 22.04 120 selenium headless-new           5
                                            chrome Ubuntu 22.04 121 selenium headless-new           4
                      fetch_GET             firefox Ubuntu 22.04 121 selenium headless              3
                      upgradeHSTS_subdomain firefox Ubuntu 22.04 121 selenium headless              2
basic     200         referrer_iframe       brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      1
parsing   200         accesswindow_direct   chrome Ubuntu 22.04 121 selenium headless-new           1
                                            edge Ubuntu 22.04 121 selenium headless-new             1
basic     200         referrer_iframe       chrome Ubuntu 22.04 121 selenium headless-new           1
parsing   200         accesswindow_direct   firefox Ubuntu 22.04 122 selenium headless              1
                      framing_iframe        safari macOS 14.3 17.3 selenium real                    1
                      imgloading_iframe     firefox Ubuntu 22.04 121 selenium headless              1
                      oac_window.open       brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      1
                      referrer_iframe       edge Ubuntu 22.04 121 selenium headless-new             1

# General Test Statistics

In [33]:
df[["browser", "resp_type", "test_name", "relation_info", "response_id", "org_scheme", "org_host", "resp_scheme", "resp_host"]].drop_duplicates().groupby(["resp_type", "browser"]).count()

test_name  \
resp_type browser                                                         
basic     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      10456   
          chrome Ubuntu 22.04 121 selenium headless-new           10456   
          edge Ubuntu 22.04 121 selenium headless-new             10456   
          firefox Ubuntu 22.04 122 selenium headless              10456   
          safari macOS 14.3 17.3 selenium real                    10456   
parsing   brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...     168774   
          chrome Ubuntu 22.04 120 selenium headless-new          163736   
          chrome Ubuntu 22.04 121 selenium headless-new          168774   
          chrome Ubuntu 22.04 122 selenium headless-new          167364   
          edge Ubuntu 22.04 121 selenium headless-new            168774   
          firefox Ubuntu 22.04 121 selenium headless             145618   
          firefox Ubuntu 22.04 122 selenium headless             168774   
          safari macOS 14.3 17.3 selenium real                   168774   

                                                              relation_info  \
resp_type browser                                                             
basic     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...          10456   
          chrome Ubuntu 22.04 121 selenium headless-new               10456   
          edge Ubuntu 22.04 121 selenium headless-new                 10456   
          firefox Ubuntu 22.04 122 selenium headless                  10456   
          safari macOS 14.3 17.3 selenium real                        10456   
parsing   brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...         168774   
          chrome Ubuntu 22.04 120 selenium headless-new              163736   
          chrome Ubuntu 22.04 121 selenium headless-new              168774   
          chrome Ubuntu 22.04 122 selenium headless-new              167364   
          edge Ubuntu 22.04 121 selenium headless-new                168774   
          firefox Ubuntu 22.04 121 selenium headless                 145618   
          firefox Ubuntu 22.04 122 selenium headless                 168774   
          safari macOS 14.3 17.3 selenium real                       168774   

                                                              response_id  \
resp_type browser                                                           
basic     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...        10456   
          chrome Ubuntu 22.04 121 selenium headless-new             10456   
          edge Ubuntu 22.04 121 selenium headless-new               10456   
          firefox Ubuntu 22.04 122 selenium headless                10456   
          safari macOS 14.3 17.3 selenium real                      10456   
parsing   brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...       168774   
          chrome Ubuntu 22.04 120 selenium headless-new            163736   
          chrome Ubuntu 22.04 121 selenium headless-new            168774   
          chrome Ubuntu 22.04 122 selenium headless-new            167364   
          edge Ubuntu 22.04 121 selenium headless-new              168774   
          firefox Ubuntu 22.04 121 selenium headless               145618   
          firefox Ubuntu 22.04 122 selenium headless               168774   
          safari macOS 14.3 17.3 selenium real                     168774   

                                                              org_scheme  \
resp_type browser                                                          
basic     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...       10456   
          chrome Ubuntu 22.04 121 selenium headless-new            10456   
          edge Ubuntu 22.04 121 selenium headless-new              10456   
          firefox Ubuntu 22.04 122 selenium headless               10456   
          safari macOS 14.3 17.3 selenium real                     10456   
parsing   brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...  

In [34]:
# Some browsers only used the more response_ids per page mode, thus these numbers don't say much
df.groupby("browser")["clean_url"].nunique()

browser
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new    95372
chrome Ubuntu 22.04 120 selenium headless-new                          21703
chrome Ubuntu 22.04 121 selenium headless-new                          95372
chrome Ubuntu 22.04 122 selenium headless-new                          21916
edge Ubuntu 22.04 121 selenium headless-new                            95372
firefox Ubuntu 22.04 121 selenium headless                             19821
firefox Ubuntu 22.04 122 selenium headless                             95372
safari macOS 14.3 17.3 selenium real                                   85492
Name: clean_url, dtype: int64

In [35]:
df.groupby("browser")["response_id"].nunique()

browser
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new    44415
chrome Ubuntu 22.04 120 selenium headless-new                          43974
chrome Ubuntu 22.04 121 selenium headless-new                          44415
chrome Ubuntu 22.04 122 selenium headless-new                          44226
edge Ubuntu 22.04 121 selenium headless-new                            44415
firefox Ubuntu 22.04 121 selenium headless                             44226
firefox Ubuntu 22.04 122 selenium headless                             44415
safari macOS 14.3 17.3 selenium real                                   44415
Name: response_id, dtype: int64

In [36]:
# Make sure that all responses actually have responses
# (create_repeat.py assumes that at least one browser collected the results)

r1 = df.groupby("label")["response_id"].nunique().sort_values()
r2 = responses.loc[responses["resp_type"] != "debug"].groupby("label")["id"].nunique().sort_values()

pd.concat([r1, r2], axis=1)

,response_id,id
label,,
CORS,6,6
CORS-ACAC,864,864
CSPvsXFO,1447,1447
CORS-ACAM,1576,1576
CORS-ACEH,1577,1577
CORS-ACAH,1577,1577
TAO,1614,1614
OAC,1713,1713
CORS-ACAO,2465,2465


In [37]:
df.groupby(["resp_type", "test_name", "relation_info"])["response_id"].nunique().to_frame()

response_id
resp_type test_name                     relation_info              
basic     accesswindow_direct           direct                   11
          fetch_GET                     credentials               6
                                        custom_headers            6
                                        simple                    6
          fetch_TEST                    custom_method             6
          framing_embed                 direct                   35
                                        nested                   35
                                        sandbox                  35
          framing_iframe                direct                   35
                                        nested                   35
                                        sandbox                  35
          framing_object                direct                   35
                                        nested                   35
                                        sandbox                  35
          fullscreen_iframe             child                    24
                                        child_allow              24
                                        child_sandbox            24
                                        direct                   24
          imgloading_iframe             direct                   17
          oac_iframe                    direct                   14
                                        sandbox                  14
          oac_window.open               window.open              14
          perfAPI_img                   direct                   13
          referrer_iframe               iframe                   16
                                        window.open              16
          script_execution_iframe       direct                   17
                                        sandbox                  17
          subresourceloadingCOEP_img    direct                   11
                                        sandbox                  11
          subresourceloadingCORP_img    direct                   11
                                        nested                   11
                                        sandbox                  11
          subresourceloadingCORP_object direct                   11
          upgradeHSTS_direct            direct                   14
          upgradeHSTS_subdomain         subdomain                14
parsing   accesswindow_direct           direct                 2656
          fetch_GET                     credentials            8059
                                        custom_headers         8059
                                        simple                 8059
          fetch_TEST                    custom_method          8059
          framing_iframe                direct                 8780
                                        nested                 8780
          fullscreen_iframe             child_allow            3880
                                        direct                 3880
          imgloading_iframe             direct                 2710
          oac_window.open               window.open            1699
          perfAPI_img                   direct                 1601
          referrer_iframe               iframe                 2797
          script_execution_iframe       direct                 2726
          subresourceloadingCOEP_img    direct                 2637
          subresourceloadingCORP_img    direct                 3357
          upgradeHSTS_direct            direct                 3324
          upgradeHSTS_subdomain         subdomain              3324

# Stability
- Basic tests:
    - 21/(10456x5) tests with different outcomes!
    - 13x firefox subresourceloadingCOEP_img (random behavior bug)
    - 6x timeout vs message 4(x edge, 2x brave)
    - 1x brave referrer different outcome
    - 1x safari COEP different outcome?
- Parsing tests:
    - 755/(168774x5) tests with different outcomes!
    - quite a lot! last time we only had 1 single test with a different outcome? (We used different versions of the browsers)
    - 3x accesswindow_direct (null vs accessible), 1x fullscreen_iframe (timeout vs message), 1x img_loading_iframe (timeout vs message)
    - All others (700+) upgradeHSTS_subdomain and upgradeHSTS_direct
        - Prevalence: `brave ~= edge > chrome >> firefox >> safari`
        - Hints at systematic issue: maybe the reset did not work or something similar?
        - Mostly timeout vs redirect=false; however some are also timeout vs redirect=false vs redirect=true!
        - Timeout maybe related to the HTTPS upgrade feature in chromium-based browsers which can take some time (>5s?)? (Non-Authoritative-Reason: HttpsUpgrades)
        - Unclear how redirect true vs false could happen? Maybe HSTS clearing did not work?
        - Manual testing always results in redirect=false for all tried clean_urls!

### Update: multiple response_ids per parsing URL test
- 2599+ test with different outcomes?!
- mostly timeouts?: Firefox fetch, other browsers subresourceloadingCOEP_img
- maybe we have too many tests per page/or the timeout is not high enough
- OT: if running `desktop_selenium.py` in the repeat mode, one has to increate `--timeout_task` (15*100 = 1500s > 1000) or decrease `--max_urls_until_restart` as we ran the repeat tests with a 3xTIMEOUT

In [127]:
# Remove all tests that timed out (test_status == 0), each test has at least one result that did not time out in each browser
df_org = df
df = df.loc[df["test_status"] == 0]

In [166]:
for group_name, resp_type_group in df.groupby("resp_type"):
    if group_name == "basic":
        continue
    print(group_name)
    stab = resp_type_group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"])[["outcome_str"]].agg(["count", "nunique"])
    # Tests with more than one outcome
    diff_outcomes = stab.loc[stab[("outcome_str", "nunique")] != 1]
    print(f"{len(diff_outcomes)} tests have different outcomes!")
    display(diff_outcomes.reset_index()["browser"].value_counts().to_frame())
    display(diff_outcomes.reset_index()[["test_name", "browser"]].value_counts().to_frame())

    diff_with_responses = diff_outcomes.reset_index().groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "browser"])["response_id"].agg(list).to_frame().reset_index()
    
    for id, (test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, browser, response_ids) in diff_with_responses.iterrows():
        # TODO: shortcut, only show the first result for each test_name/browser; results are usually always the same!
        print(test_name, response_ids)
        rows = df.loc[(df["org_scheme"] == org_scheme) & (df["test_name"] == test_name) & (df["resp_scheme"] == resp_scheme) & (df["resp_host"] == resp_host) & (df["relation_info"] == relation_info) & (df["response_id"].isin(response_ids)) & (df["browser"] == browser)]
        with pd.option_context("display.max_colwidth", 200):
            display(rows[["outcome_str", "browser"]].value_counts().to_frame())
            disp = rows.drop_duplicates(subset="outcome_str")[["browser", "outcome_str", "raw_header", "test_name", "org_scheme", "org_host", "resp_scheme", "relation_info", "resp_host", "full_url"]]
            disp = disp.style.format({'full_url': make_clickable})
            display(disp)
        # input("Continue!")

parsing
2599 tests have different outcomes!


,count
browser,
firefox Ubuntu 22.04 122 selenium headless,967
edge Ubuntu 22.04 121 selenium headless-new,629
chrome Ubuntu 22.04 121 selenium headless-new,514
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,483
safari macOS 14.3 17.3 selenium real,6


count
(test_name, )              (browser, )                                              
fetch_GET                  firefox Ubuntu 22.04 122 selenium headless            754
subresourceloadingCOEP_img chrome Ubuntu 22.04 121 selenium headless-new         277
upgradeHSTS_subdomain      brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...    262
subresourceloadingCOEP_img edge Ubuntu 22.04 121 selenium headless-new           238
upgradeHSTS_subdomain      edge Ubuntu 22.04 121 selenium headless-new           234
upgradeHSTS_direct         brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...    219
                           edge Ubuntu 22.04 121 selenium headless-new           137
fetch_TEST                 firefox Ubuntu 22.04 122 selenium headless            106
upgradeHSTS_subdomain      chrome Ubuntu 22.04 121 selenium headless-new          89
upgradeHSTS_direct         chrome Ubuntu 22.04 121 selenium headless-new          74
imgloading_iframe          chrome Ubuntu 22.04 121 selenium headless-new          71
subresourceloadingCOEP_img firefox Ubuntu 22.04 122 selenium headless             47
upgradeHSTS_direct         firefox Ubuntu 22.04 122 selenium headless             45
imgloading_iframe          edge Ubuntu 22.04 121 selenium headless-new            18
upgradeHSTS_subdomain      firefox Ubuntu 22.04 122 selenium headless             15
upgradeHSTS_direct         safari macOS 14.3 17.3 selenium real                    3
upgradeHSTS_subdomain      safari macOS 14.3 17.3 selenium real                    2
accesswindow_direct        edge Ubuntu 22.04 121 selenium headless-new             2
                           chrome Ubuntu 22.04 121 selenium headless-new           1
fullscreen_iframe          chrome Ubuntu 22.04 121 selenium headless-new           1
                           brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      1
perfAPI_img                chrome Ubuntu 22.04 121 selenium headless-new           1
script_execution_iframe    brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      1
imgloading_iframe          safari macOS 14.3 17.3 selenium real                    1

accesswindow_direct [22059, 22478]


,,count
outcome_str,browser,
{'window.open.opener': 'null'},edge Ubuntu 22.04 121 selenium headless-new,4
"{'window.open.opener': 'object ""[object Window]""'}",edge Ubuntu 22.04 121 selenium headless-new,2


...

upgradeHSTS_subdomain [31730, 31932, 32023, 32123, 32131, 32149, 32222, 32264, 32298, 32340, 32460, 32502, 32559, 32658, 32753]


,,count
outcome_str,browser,
{'response.redirected': False},firefox Ubuntu 22.04 122 selenium headless,30
Fetch timed out,firefox Ubuntu 22.04 122 selenium headless,15


,browser,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
263665,firefox Ubuntu 22.04 122 selenium headless,{'response.redirected': False},"[['strict-transport-security', 'max-a?ge=60']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=parsing&browser_id=1&label=HSTS&first_id=32149&last_id=32149&scheme=http
303880,firefox Ubuntu 22.04 122 selenium headless,Fetch timed out,"[['strict-transport-security', 'max-a?ge=60']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=parsing&browser_id=1&label=HSTS&first_id=32149&last_id=32149&scheme=http


upgradeHSTS_subdomain [29963, 33015]


KeyboardInterrupt: 

## More stability?!
- How many outcomes did we collect per test in the different browsers? Also shows tests with only one outcome
- How many of them are different?

In [63]:
final_df = pd.DataFrame()

for group, d in df.groupby("browser"):
    # Calculate counts and unique counts
    aggs = ["count"] # ["count", "nunique"]
    res = d.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])["outcome_str"].agg(aggs)
    # Get value counts for each combination of counts and unique counts
    counts = res.value_counts().to_frame()
    counts = counts.rename(columns={"count": group})

    only_once = res.loc[res["count"] == 1]
    if len(only_once) < 200:
        print(group)
        display(only_once.reset_index()[["test_name", "relation_info"]].value_counts().to_frame())

    final_df = pd.concat([final_df, counts], axis=1)

display(final_df)

brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new


count
test_name       relation_info       
oac_window.open window.open       13
oac_iframe      direct            10
                sandbox           10
referrer_iframe iframe             5
                window.open        5

chrome Ubuntu 22.04 121 selenium headless-new


count
test_name       relation_info       
framing_embed   nested             3
                sandbox            3
framing_object  sandbox            3
framing_iframe  nested             3
framing_object  nested             3
framing_embed   direct             2
framing_iframe  sandbox            2
                direct             2
framing_object  direct             2
referrer_iframe iframe             1

edge Ubuntu 22.04 121 selenium headless-new


...

firefox Ubuntu 22.04 122 selenium headless


,,count
test_name,relation_info,
referrer_iframe,iframe,8


safari macOS 14.3 17.3 selenium real


count
test_name           relation_info       
referrer_iframe     iframe            30
                    window.open       14
accesswindow_direct direct            13
perfAPI_img         direct             8

,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,safari macOS 14.3 17.3 selenium real
count,,,,,,,,
3,168641.0,NaN,96479.0,NaN,163456.0,NaN,168771.0,36657
2,10546.0,NaN,10433.0,NaN,15665.0,11.0,10451.0,141733
1,43.0,163731.0,24.0,167364.0,23.0,145584.0,8.0,65
4,NaN,NaN,72294.0,NaN,86.0,NaN,NaN,744
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31


In [77]:
grouped = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser", "resp_type"])['outcome_str'].count().reset_index()

with pd.option_context("display.max_rows", 202):
    res = grouped.groupby(["browser", "resp_type", "test_name"])["outcome_str"].agg(["mean", "count", "sum"]).sort_values(by="mean", ascending=False)
    display(res.describe())
    display(res.loc[res["mean"] > 0])

,mean,count,sum
count,202.000000,202.000000,202.000000
mean,2.101130,6796.237624,15095.118812
std,0.734567,10940.399341,28169.426613
min,1.000000,28.000000,56.000000
25%,1.998535,448.000000,881.000000
50%,2.000000,3398.000000,5312.000000
75%,2.999199,6648.000000,16149.500000
max,3.500000,48354.000000,169239.000000


mean  \
browser                                            resp_type test_name                                 
chrome Ubuntu 22.04 121 selenium headless-new      parsing   fetch_GET                      3.500000   
                                                             fetch_TEST                     3.500000   
                                                             referrer_iframe                3.500000   
                                                             framing_iframe                 3.500000   
                                                             perfAPI_img                    3.500000   
                                                             script_execution_iframe        3.500000   
                                                             oac_window.open                3.500000   
                                                             fullscreen_iframe              3.499742   
                                                             subresourceloadingCORP_img     3.490766   
                                                             subresourceloadingCOEP_img     3.415813   
                                                             imgloading_iframe              3.078782   
                                                             accesswindow_direct            3.000565   
edge Ubuntu 22.04 121 selenium headless-new        parsing   perfAPI_img                    3.000000   
                                                             oac_window.open                3.000000   
                                                             upgradeHSTS_subdomain          3.000000   
                                                             upgradeHSTS_direct             3.000000   
                                                             script_execution_iframe        3.000000   
                                                             subresourceloadingCOEP_img     3.000000   
                                                             subresourceloadingCORP_img     3.000000   
firefox Ubuntu 22.04 122 selenium headless         parsing   imgloading_iframe              3.000000   
                                                             script_execution_iframe        3.000000   
                                                             referrer_iframe                3.000000   
                                                             perfAPI_img                    3.000000   
                                                             oac_window.open                3.000000   
chrome Ubuntu 22.04 121 selenium headless-new      parsing   upgradeHSTS_subdomain          3.000000   
                                                             upgradeHSTS_direct             3.000000   
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... parsing   perfAPI_img                    3.000000   
                                                             fullscreen_iframe              3.000000   
firefox Ubuntu 22.04 122 selenium headless         parsing   fetch_TEST                     3.000000   
                                                             fetch_GET                      3.000000   
                                                             upgradeHSTS_subdomain          3.000000   
                                                             upgradeHSTS_direct             3.000000   
                                                             subresourceloadingCOEP_img     3.000000   
                                                             subresourceloadingCORP_img     3.000000   
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... parsing   referrer_iframe                3.000000   
                                                             subresourceloadingCOEP_img     3.000000   
                                                             framing_iframe                 2.999858   
edge Ubuntu 22.04 121 selenium headless-

In [74]:
# Different outcomes (in percentage for a test group (test_name)
grouped = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser", "resp_type"])['outcome_str'].nunique().reset_index()

grouped["diff_outcome"] = grouped["outcome_str"] > 1
with pd.option_context("display.max_rows", 50):
    res = grouped.groupby(["browser", "resp_type", "test_name"])["diff_outcome"].agg(["mean", "count", "sum"]).sort_values(by="mean", ascending=False)
    display(res.describe())
    display(res.loc[res["mean"] > 0])

,mean,count,sum
count,202.000000,202.000000,202.000000
mean,0.001928,6796.237624,12.970297
std,0.007695,10940.399341,66.642800
min,0.000000,28.000000,0.000000
25%,0.000000,448.000000,0.000000
50%,0.000000,3398.000000,0.000000
75%,0.000000,6648.000000,0.000000
max,0.052522,48354.000000,754.000000


mean  \
browser                                            resp_type test_name                              
chrome Ubuntu 22.04 121 selenium headless-new      parsing   subresourceloadingCOEP_img  0.052522   
edge Ubuntu 22.04 121 selenium headless-new        parsing   subresourceloadingCOEP_img  0.045127   
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... parsing   upgradeHSTS_subdomain       0.039410   
firefox Ubuntu 22.04 122 selenium headless         basic     subresourceloadingCOEP_img  0.036932   
edge Ubuntu 22.04 121 selenium headless-new        basic     upgradeHSTS_subdomain       0.035714   
                                                   parsing   upgradeHSTS_subdomain       0.035199   
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... parsing   upgradeHSTS_direct          0.032942   
edge Ubuntu 22.04 121 selenium headless-new        parsing   upgradeHSTS_direct          0.020608   
firefox Ubuntu 22.04 122 selenium headless         parsing   fetch_GET                   0.015593   
chrome Ubuntu 22.04 121 selenium headless-new      parsing   upgradeHSTS_subdomain       0.013387   
                                                             imgloading_iframe           0.013100   
                                                             upgradeHSTS_direct          0.011131   
firefox Ubuntu 22.04 122 selenium headless         parsing   subresourceloadingCOEP_img  0.008912   
                                                             upgradeHSTS_direct          0.006769   
                                                             fetch_TEST                  0.006576   
edge Ubuntu 22.04 121 selenium headless-new        parsing   imgloading_iframe           0.003321   
safari macOS 14.3 17.3 selenium real               basic     subresourceloadingCOEP_img  0.002841   
firefox Ubuntu 22.04 122 selenium headless         parsing   upgradeHSTS_subdomain       0.002256   
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... basic     referrer_iframe             0.001953   
                                                             framing_object              0.000595   
edge Ubuntu 22.04 121 selenium headless-new        basic     framing_object              0.000595   
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... basic     framing_embed               0.000595   
edge Ubuntu 22.04 121 selenium headless-new        basic     framing_iframe              0.000595   
                                                             framing_embed               0.000595   
safari macOS 14.3 17.3 selenium real               parsing   upgradeHSTS_direct          0.000451   
edge Ubuntu 22.04 121 selenium headless-new        parsing   accesswindow_direct         0.000377   
chrome Ubuntu 22.04 121 selenium headless-new      parsing   perfAPI_img                 0.000312   
safari macOS 14.3 17.3 selenium real               parsing   upgradeHSTS_subdomain       0.000301   
chrome Ubuntu 22.04 121 selenium headless-new      parsing   accesswindow_direct         0.000188   
safari macOS 14.3 17.3 selenium real               parsing   imgloading_iframe           0.000185   
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... parsing   script_execution_iframe     0.000183   
                                                             fullscreen_iframe           0.000064   
chrome Ubuntu 22.04 121 selenium headless-new      parsing   fullscreen_iframe           0.000064   

                                                                                         count  \
browser                                            resp_type test_name                           
chrome Ubuntu 22.04 121 selenium headless-new      parsing   subresourceloadingCOEP_img   5274   
edge Ubuntu 22.04 121 selenium headless-new        parsing   subresourceloadingCOEP_img   5274   
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... parsing   upgradeHSTS_subdomain        6648   
firefox Ubuntu 22.04 122 selenium headless         basic     subres

In [56]:
# Run the test groups with many different outcomes again?
# Groups with non-negligble different outcomes are subresourceloadingCOEP_img, upgradeHSTS_subdomain, upgradeHSTS_direct, imgloading_iframe, fetch_TEST, fetch_GET
test_files_to_repeat =  ["fetch-cors.sub.html", "subresource-loading-coep.sub.html", "subresource-loading-csp.sub.html", "upgrade-hsts.sub.html"]

# TODO: implement utils.py/desktop_selenium mode to only rerun these files?!
# For now, simply run everything again to get some more results for stability?!

In [93]:
# Majority voting for the tests with multiple outcomes
df.sample(10000).groupby(["test_name", "relation_info", "browser", "org_host", "resp_scheme", "org_origin", "resp_origin", "response_id"])["outcome_str"].agg([pd.Series.mode, "nunique", "count"]).reset_index().sort_values("count")

,test_name,relation_info,browser,org_origin,resp_origin,response_id,mode,nunique,count
6644,fullscreen_iframe,child_allow,safari macOS 14.3 17.3 selenium real,http://sub.headers.websec.saarland,https://headers.webappsec.eu,37738,message timeout,1,1
6635,fullscreen_iframe,child_allow,safari macOS 14.3 17.3 selenium real,http://sub.headers.websec.saarland,https://headers.webappsec.eu,36709,message timeout,1,1
6636,fullscreen_iframe,child_allow,safari macOS 14.3 17.3 selenium real,http://sub.headers.websec.saarland,https://headers.webappsec.eu,36710,message timeout,1,1
6637,fullscreen_iframe,child_allow,safari macOS 14.3 17.3 selenium real,http://sub.headers.websec.saarland,https://headers.webappsec.eu,36757,message timeout,1,1
6638,fullscreen_iframe,child_allow,safari macOS 14.3 17.3 selenium real,http://sub.headers.websec.saarland,https://headers.webappsec.eu,36899,message timeout,1,1
...,...,...,...,...,...,...,...,...,...
6788,fullscreen_iframe,direct,chrome Ubuntu 22.04 121 selenium headless-new,http://sub.headers.websec.saarland,https://headers.webappsec.eu,35868,fullscreenEnabled: true,1,2
2527,fetch_GET,simple,chrome Ubuntu 22.04 121 selenium headless-new,https://sub.headers.websec.saarland,https://headers.webappsec.eu,13355,"{'error': 'null', 'headers': 'content-length,t...",1,2
8827,subresourceloadingCORP_img,direct,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) ...,http://sub.headers.websec.saarland,http://headers.websec.saarland,52,load,1,2
8608,subresourceloadingCOEP_img,direct,chrome Ubuntu 22.04 121 selenium headless-new,http://sub.headers.websec.saarland,https://headers.webappsec.eu,21362,"{'image-events': {'swag.jpg': 'load', 'swag-sa...",1,2


# Browser differences!

In [171]:
df = df_org
df = df.loc[df["test_status"] == 0]

In [172]:
df_stab = df

# Only keep one row for each test
# If the test had non-deterministic results, use majority voting! (only works reliably if there a enough repetitions of the tests runs)

print("Original data entries", len(df))
#df = df.drop_duplicates(subset=["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser", "outcome_str"])
#print("Only one row for deterministic tests", len(df))

# Majority voting (pd.Series.mode returns the most frequest item) (quite slow)
#df_mult_outcomes = df[df.duplicated(subset=["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"], keep=False)]
df = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"])["outcome_str"].agg(pd.Series.mode).apply(lambda x: x[0]).reset_index()
#df = pd.concat([df[~df.duplicated(subset=["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"], keep=False)], df_mult_outcomes])
print("Only one row for each test (per browser; majority voting)", len(df))

Original data entries 3049214
Only one row for each test (per browser; majority voting) 1372840


In [136]:
# Most tests have exactly one outcome, quite some have two, couple have three!
res = df.groupby(["resp_type", "test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
res = res["outcome_str"].agg("nunique")
display(res.value_counts().to_frame())

,count
outcome_str,
1,168925
2,10218
3,87


In [137]:
# Basic:
# subresourceloadingCORP_object has different results for 100% of tests
# couple of others have 20%+ differences
# coupel has 0%

# Parsing:
# oac_window.open has 99%
# Other test groups have between 0.5 - 7.5% of tests with different outcomes
r = res.reset_index()
r["diff"] = r["outcome_str"] != 1
r.groupby(["resp_type", "test_name", "relation_info"])["diff"].agg(["count", "sum", "mean"]).sort_values("mean", ascending=False)

count   sum      mean
resp_type test_name                     relation_info                        
basic     subresourceloadingCORP_object direct            176   176  1.000000
parsing   oac_window.open               window.open      3398  3370  0.991760
basic     oac_window.open               window.open       224   192  0.857143
          oac_iframe                    direct            224   144  0.642857
                                        sandbox           224   144  0.642857
          referrer_iframe               window.open       256   117  0.457031
          fullscreen_iframe             direct            384   142  0.369792
          framing_embed                 direct            560   186  0.332143
          framing_object                direct            560   186  0.332143
          fullscreen_iframe             child_allow       384   113  0.294271
          perfAPI_img                   direct            208    55  0.264423
          subresourceloadingCOEP_img    direct            176    39  0.221591
          framing_object                sandbox           560   120  0.214286
                                        nested            560   120  0.214286
          framing_embed                 nested            560   120  0.214286
                                        sandbox           560   120  0.214286
          accesswindow_direct           direct            176    28  0.159091
parsing   subresourceloadingCOEP_img    direct           5274   689  0.130641
          imgloading_iframe             direct           5420   540  0.099631
basic     script_execution_iframe       sandbox           272    21  0.077206
          referrer_iframe               iframe            256    18  0.070312
parsing   fetch_GET                     custom_headers  16118  1067  0.066199
basic     fullscreen_iframe             child             384    23  0.059896
parsing   upgradeHSTS_direct            direct           6648   377  0.056709
basic     imgloading_iframe             direct            272    13  0.047794
parsing   fullscreen_iframe             child_allow      7760   355  0.045747
          script_execution_iframe       direct           5452   216  0.039618
basic     upgradeHSTS_direct            direct             28     1  0.035714
parsing   fullscreen_iframe             direct           7760   272  0.035052
          upgradeHSTS_subdomain         subdomain        6648   181  0.027226
          framing_iframe                direct          17560   366  0.020843
          referrer_iframe               iframe           5594   102  0.018234
basic     subresourceloadingCOEP_img    sandbox           176     3  0.017045
          framing_iframe                sandbox           560     8  0.014286
parsing   fetch_TEST                    custom_method   16118   192  0.011912
          perfAPI_img                   direct           3202    33  0.010306
          subresourceloadingCORP_img    direct           6714    68  0.010128
          framing_iframe                nested          17560   154  0.008770
          accesswindow_direct           direct           5312    40  0.007530
          fetch_GET                     simple          16118   106  0.006576
basic     subresourceloadingCORP_img    direct            176     1  0.005682
parsing   fetch_GET                     credentials     16118    87  0.005398
basic     fetch_GET                     credentials        96     0  0.000000
                                        simple             96     0  0.000000
                                        custom_headers     96     0  0.000000
          fullscreen_iframe             child_sandbox     384     0  0.000000
          framing_iframe                nested            560     0  0.000000
                                        direct            560     0  0.000000
          fetch_TEST                    custom_method      96     0  0.000000
          upgradeHSTS_subdomain         subdomain          28     0  0.000000


In [138]:
# Select only test rows with more than one outcome
tests_with_mult_outcomes = r.loc[r["diff"]].set_index(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
df_mult = df.loc[df[["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"]].apply(tuple, axis=1).isin(tests_with_mult_outcomes.index)]

print(f"All test rows: {len(df)}, Rows with more than one outcome: {len(df_mult)}")

All test rows: 1372840, Rows with more than one outcome: 74620


In [139]:
# Display difference groups
for grouping, group in df_mult.groupby(["test_name", "relation_info"]):
    print(grouping)
    df_list = []
    for _, one_test in group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"]):
        browsers = one_test.groupby("outcome_str")["browser"].unique().apply(list).apply(sorted).to_frame()
        new = True
        for df_b in df_list:
            if df_b.equals(browsers):
                new = False
        if new:
            df_list.append(browsers)
    print(len(df_list))
    for df_b in df_list:
        with pd.option_context("display.max_colwidth", 1000):
            display(df_b)
    #input("Continue!")

('accesswindow_direct', 'direct')
6


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new]"
"{'window.open.opener': 'object ""[object Window]""'}","[firefox Ubuntu 22.04 122 selenium headless, safari macOS 14.3 17.3 selenium real]"


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless]"
"{'window.open.opener': 'object ""[object Window]""'}",[safari macOS 14.3 17.3 selenium real]


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, safari macOS 14.3 17.3 selenium real]"
"{'window.open.opener': 'object ""[object Window]""'}","[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless]"


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless]"
"{'window.open.opener': 'object ""[object Window]""'}",[safari macOS 14.3 17.3 selenium real]


...

,browser
outcome_str,
Fetch timed out,[chrome Ubuntu 22.04 122 selenium headless-new]
{'response.redirected': False},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, safari macOS 14.3 17.3 selenium real]"


,browser
outcome_str,
Fetch timed out,[firefox Ubuntu 22.04 121 selenium headless]
{'response.redirected': False},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, safari macOS 14.3 17.3 selenium real]"


,browser
outcome_str,
Fetch timed out,"[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless]"
{'response.redirected': True},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, safari macOS 14.3 17.3 selenium real]"


,browser
outcome_str,
{'response.redirected': False},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new]"
{'response.redirected': True},"[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, safari macOS 14.3 17.3 selenium real]"


,browser
outcome_str,
{'response.redirected': False},"[chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless]"
{'response.redirected': True},[safari macOS 14.3 17.3 selenium real]


In [140]:
# Other approach to list the difference groups
def get_uniques(df):
    browsers = f"{df['name'].unique().tolist()}"
    try:
        outcome = unique_outcomes[df.name[0]]
    except KeyError:
        outcome = {}
        #outcome = []
    outcome[browsers] = df.name[1]
    #outcome.append(f"{df.name[1]} ({browsers})")
    unique_outcomes[df.name[0]] = outcome

for grouping, group in df_mult.groupby(["test_name", "relation_info"]):
    
    unique_outcomes = {}
    group.groupby(["name"])["outcome_str"].value_counts(normalize=False).reset_index().groupby(["outcome_str", "count"], group_keys=True).apply(get_uniques)
    d = pd.DataFrame.from_dict(unique_outcomes, orient="index")
    if d.shape[1] != 1:
        display(d)
        print()

,['safari'],['firefox'],['edge'],['brave'],['chrome'],"['brave', 'edge']"
{'window.open.opener': 'null'},16,48,54.0,56.0,111,NaN
"{'window.open.opener': 'object ""[object Window]""'}",52,60,NaN,NaN,36,12.0


,"['brave', 'edge']","['chrome', 'safari']",['firefox'],['safari'],['chrome']
"{'error': 'null', 'headers': 'content-length,'}",3.0,9.0,20,NaN,NaN
"{'error': 'null', 'headers': 'content-length,test,'}",19.0,NaN,40,31.0,57.0
"{'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}",65.0,NaN,30,47.0,193.0
"{'error': 'object ""AbortError: Fetch is aborted""', 'headers': ''}",NaN,NaN,78,NaN,NaN


,['firefox'],"['brave', 'edge', 'safari']",['chrome'],"['brave', 'edge']",['safari']
"{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,server,test,'}",2,6.0,18.0,NaN,NaN
"{'error': 'null', 'headers': 'content-length,'}",154,NaN,855.0,285.0,291.0
"{'error': 'null', 'headers': 'content-length,test,'}",218,NaN,1287.0,429.0,438.0
"{'error': 'object ""AbortError: Fetch is aborted""', 'headers': ''}",1045,NaN,NaN,NaN,40.0
"{'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}",211,NaN,1039.0,347.0,292.0


,"['brave', 'edge', 'safari']",['chrome'],"['brave', 'edge', 'firefox', 'safari']",['firefox'],"['brave', 'edge']",['safari']
"{'error': 'null', 'headers': ''}",3.0,9.0,NaN,NaN,NaN,NaN
"{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,location,server,test,'}",1.0,3.0,NaN,NaN,NaN,NaN
"{'error': 'null', 'headers': 'cache-control,content-type,'}",2.0,6.0,NaN,17.0,NaN,NaN
"{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,server,test,'}",NaN,12.0,4.0,NaN,NaN,NaN
"{'error': 'null', 'headers': 'content-length,'}",NaN,12.0,NaN,28.0,4.0,10.0
"{'error': 'null', 'headers': 'content-length,test,'}",NaN,75.0,NaN,52.0,25.0,40.0
"{'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}",NaN,199.0,NaN,24.0,67.0,46.0
"{'error': 'object ""AbortError: Fetch is aborted""', 'headers': ''}",NaN,NaN,NaN,80.0,NaN,NaN


,['firefox'],"['brave', 'edge', 'safari']",['chrome'],"['brave', 'edge']",['safari'],"['firefox', 'safari']"
"{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,server,test,'}",3.0,5.0,15.0,NaN,NaN,NaN
"{'error': 'null', 'headers': 'content-length,'}",53.0,NaN,135.0,45.0,51.0,NaN
"{'error': 'object ""AbortError: Fetch is aborted""', 'headers': ''}",168.0,NaN,NaN,NaN,40.0,NaN
"{'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}",48.0,NaN,226.0,76.0,21.0,NaN
"{'error': 'null', 'headers': 'content-length,test,'}",NaN,NaN,198.0,66.0,NaN,75.0


...

,['safari'],"['brave', 'chrome', 'edge', 'firefox']"
"{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",3.0,NaN
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",NaN,3.0


,['safari'],"['brave', 'edge']",['firefox'],['chrome']
error,17,52.0,57.0,156.0
load,40,17.0,80.0,49.0
timeout: no event fired,12,NaN,NaN,NaN


,"['brave', 'chrome', 'edge']",['firefox'],['safari']
error,44.0,76.0,NaN
load,132.0,100.0,NaN
timeout: no event fired,NaN,NaN,176.0


,['chrome'],['firefox'],['edge'],['brave'],['safari']
Fetch timed out,38,63,NaN,NaN,NaN
TypeError: Failed to fetch,79,18,26.0,27.0,NaN
{'response.redirected': False},331,639,123.0,120.0,98.0
{'response.redirected': True},641,32,217.0,215.0,280.0


,['firefox'],['chrome'],['brave'],['edge'],['safari']
Fetch timed out,43,60,NaN,NaN,NaN
TypeError: Failed to fetch,18,79,26.0,27.0,NaN
{'response.redirected': False},291,267,111.0,113.0,120.0
{'response.redirected': True},8,111,32.0,37.0,61.0


In [173]:
# Function to calculate absolute and percentage overlap
def calculate_overlap(df, col1, col2):
    df = df[[col1, col2]].dropna()
    df.columns.values[0] = 0
    df.columns.values[1] = 1
    absolute_overlap = (df[0] == df[1]).sum()
    unique_rows = len(df)
    diff_rows = unique_rows - absolute_overlap
    percentage_diff = (diff_rows / unique_rows) * 100
    return diff_rows, percentage_diff, unique_rows


def display_overlap(df, show_all=True, name="All"):
    print(name)
    sim_frame = df.set_index(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"])["outcome_str"].unstack().astype("str")
    sim_frame = sim_frame.reset_index().drop(columns=["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
    # Create a dictionary to store results
    results = {}
    matrix = {}
    
    # Iterate through all pairs of columns
    for i, col1 in enumerate(sim_frame.columns):
        for col2 in sim_frame.columns:
            key = f"{col1}_vs_{col2}"
            results[key] = calculate_overlap(sim_frame, col1, col2)
            try:
                l = matrix[col1]
            except KeyError:
                l = {}
            # 0 for absolute count of differences, 1 for percentage
            l[col2] = results[key][0]
            matrix[col1] = l
            
    
    # Convert the results to a DataFrame for better visualization
    results_df = pd.DataFrame.from_dict(results, orient='index', columns=['Absolute Diff', 'Percentage Diff', "Count"])
    
    # Display the results
    if show_all:
        display(results_df)
    
    # Display difference matrix
    with pd.option_context("display.max_columns", 28):
        if show_all:
            display(pd.DataFrame(matrix))
        else:
            display(pd.DataFrame(matrix).drop_duplicates().T)


display_overlap(df)

for name, group in df.groupby("test_name"):
    display_overlap(group, show_all=False, name=name)

All


,Absolute Diff,Percentage Diff,Count
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new_vs_brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,0.000000,179230
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new_vs_chrome Ubuntu 22.04 120 selenium headless-new,15742,8.783128,179230
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new_vs_chrome Ubuntu 22.04 121 selenium headless-new,8,0.004464,179230
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new_vs_chrome Ubuntu 22.04 122 selenium headless-new,12757,7.117670,179230
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new_vs_edge Ubuntu 22.04 121 selenium headless-new,12,0.006695,179230
...,...,...,...
safari macOS 14.3 17.3 selenium real_vs_chrome Ubuntu 22.04 122 selenium headless-new,12960,7.230932,179230
safari macOS 14.3 17.3 selenium real_vs_edge Ubuntu 22.04 121 selenium headless-new,459,0.256096,179230
safari macOS 14.3 17.3 selenium real_vs_firefox Ubuntu 22.04 121 selenium headless,34025,18.983987,179230
safari macOS 14.3 17.3 selenium real_vs_firefox Ubuntu 22.04 122 selenium headless,525,0.292920,179230


,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,safari macOS 14.3 17.3 selenium real
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,15742,8,12757,12,34119,554,459
chrome Ubuntu 22.04 120 selenium headless-new,15742,0,15737,7364,15741,27480,16168,15939
chrome Ubuntu 22.04 121 selenium headless-new,8,15737,0,12752,8,34116,546,451
chrome Ubuntu 22.04 122 selenium headless-new,12757,7364,12752,0,12756,25389,13180,12960
edge Ubuntu 22.04 121 selenium headless-new,12,15741,8,12756,0,34120,554,459
firefox Ubuntu 22.04 121 selenium headless,34119,27480,34116,25389,34120,0,33695,34025
firefox Ubuntu 22.04 122 selenium headless,554,16168,546,13180,554,33695,0,525
safari macOS 14.3 17.3 selenium real,459,15939,451,12960,459,34025,525,0


accesswindow_direct


,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Ubuntu 22.04 120 selenium headless-new
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,176
chrome Ubuntu 22.04 120 selenium headless-new,176,0
chrome Ubuntu 22.04 121 selenium headless-new,0,176
chrome Ubuntu 22.04 122 selenium headless-new,176,0
edge Ubuntu 22.04 121 selenium headless-new,0,176
firefox Ubuntu 22.04 121 selenium headless,176,0
firefox Ubuntu 22.04 122 selenium headless,0,176
safari macOS 14.3 17.3 selenium real,0,176


...

,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,firefox Ubuntu 22.04 122 selenium headless,safari macOS 14.3 17.3 selenium real
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,32,176
chrome Ubuntu 22.04 121 selenium headless-new,0,32,176
edge Ubuntu 22.04 121 selenium headless-new,0,32,176
firefox Ubuntu 22.04 122 selenium headless,32,0,176
safari macOS 14.3 17.3 selenium real,176,176,0


upgradeHSTS_direct


,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,safari macOS 14.3 17.3 selenium real
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,542,1,58,96,32,31
chrome Ubuntu 22.04 120 selenium headless-new,542,0,541,540,577,570,568
chrome Ubuntu 22.04 121 selenium headless-new,1,541,0,57,97,31,30
chrome Ubuntu 22.04 122 selenium headless-new,58,540,57,0,98,88,87
edge Ubuntu 22.04 121 selenium headless-new,1,541,0,57,97,31,30
firefox Ubuntu 22.04 121 selenium headless,96,577,97,98,0,68,91
firefox Ubuntu 22.04 122 selenium headless,32,570,31,88,68,0,25
safari macOS 14.3 17.3 selenium real,31,568,30,87,91,25,0


upgradeHSTS_subdomain


,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,safari macOS 14.3 17.3 selenium real
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,551,4,77,6,82,34,33
chrome Ubuntu 22.04 120 selenium headless-new,551,0,547,559,548,563,575,573
chrome Ubuntu 22.04 121 selenium headless-new,4,547,0,73,2,78,30,29
chrome Ubuntu 22.04 122 selenium headless-new,77,559,73,0,74,95,103,102
edge Ubuntu 22.04 121 selenium headless-new,6,548,2,74,0,79,32,31
firefox Ubuntu 22.04 121 selenium headless,82,563,78,95,79,0,49,71
firefox Ubuntu 22.04 122 selenium headless,34,575,30,103,32,49,0,23
safari macOS 14.3 17.3 selenium real,33,573,29,102,31,71,23,0


In [ ]:
# How many rows exist for tree creation
df[["test_name", "label", "relation_info"]].value_counts()

In [ ]:
h2o.init(nthreads=50, max_mem_size="100G", log_level="WARN")
h2o.no_progress()


# Limit on responses that have more than one outcome!
# This will remove all the branches that are the same in all browsers (and make trees large and confusing)
condition = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])["outcome_str"].transform("nunique") != 1
tree_df = df.loc[condition]
print("Remove tests that are the same in all browsers", len(tree_df))
base_dir = f"trees/{datetime.today().strftime('%Y-%m-%dT%H:%M')}" 

for group_name, group in tree_df.groupby(["test_name", "relation_info"]):
    print(group_name)
    pred_cols = ["browser", "org_origin", "resp_origin", "status_code", "raw_header"]
    group = group[["outcome_str", *pred_cols]]
    tree = make_tree(group, pred_cols, group_name, base_dir)
    # print(tree)


# Manual analysis

In [29]:
# Chromium-based treats host-source as case sensitive even though it should not? (URLs and CSP is case-insensitive!)
# Play around: https://observer.sectec.rocks/opg/iframe/?url=https://echo.sectec.rocks/echo/?content-security-policy=img-src%20HTTPS://echo.sectec.rocks&ecocnt_css=%3Cimg%20src=https://echo.sectec.rocks/%3E&content-type=text/html
# Issue is in the scheme match which should be case-insensitive but is in Chromium-based browsers https://w3c.github.io/webappsec-csp/#match-schemes
# host-part matching was fixed here https://issues.chromium.org/issues/41412675, seems like they forgot fixing scheme-sensitivity 
df.loc[df["raw_header"].str.contains("img-src HTTPS://")][["clean_url", "raw_header", "outcome_str", "browser"]].value_counts().to_frame()

count
clean_url                                          raw_header                                         outcome_str browser                                                  
http://sub.headers.websec.saarland/_hp/tests/su... [['content-security-policy', 'img-src HTTPS://S... error       brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      1
                                                                                                                  chrome Ubuntu 22.04 121 selenium headless-new           1
                                                                                                                  edge Ubuntu 22.04 121 selenium headless-new             1
                                                                                                                  firefox Ubuntu 22.04 122 selenium headless              1
                                                                                                                  safari macOS 14.3 17.3 selenium real                    1
https://sub.headers.websec.saarland/_hp/tests/s... [['content-security-policy', 'img-src HTTPS://S... error       brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      1
                                                                                                                  chrome Ubuntu 22.04 121 selenium headless-new           1
                                                                                                                  edge Ubuntu 22.04 121 selenium headless-new             1
                                                                                                      load        firefox Ubuntu 22.04 122 selenium headless              1
                                                                                                                  safari macOS 14.3 17.3 selenium real                    1